# 01/07

# get_antennas para mexico
Solo voy a mirar la data procesada Pospaga!

En los raw CDRs sucede que las antennas no tienen otra identificacion mas que por su lat/long. Luego hay que extraer todos los pares de lat/long para analizar la "viabilidad" del par i.e. ver que esten dentor de mexico y que no se caigan al mar por ejemplo, para luego aplicar eso en el dataset. El id de la antenna seguira siendo su propiedad (el par lat/long), solamente que en algun momento se hara la transformacion.

La idea es recorrer TODOS los raw cdr
## definiciones utilizadas

* la fecha viene dado en DD/MM/YYY HH:MM:SS (am/pm)
* *Time* columns are measured in seconds.

### el sframe procesado vendria con este header
    USER OTHER_USER DIRECTION DURATION	LATITUDE	LONGITUDE DATE

Direction es quien llama a quien (saliente es 1-->2, viceversa entrante). Duracion medido en segundos y lat long estan en float datatype

In [3]:
%autosave 180
import numpy as np; import os;import random;
import graphlab as gl
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
np.random.seed(2016)

rootdir="/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/"
os.chdir(rootdir)

Autosaving every 180 seconds


In [4]:
direc = rootdir
!ls $direc
!du -sh $direc

2015
191G	/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/


In [ ]:
cd "2015/"

In [9]:
#ejemplo  /grandata/voice/201505_geo/Geo_Voz_Pospago_27-280515.txt.gz"
#seteamos el lugar de trabajo

year = 2015; 
months= [5,6,7];

def get_input_file(month,year = 2015,plan = "Pospago"):
    return rootdir +"{y}/{m:0=2d}/{pl}"\
                .format(y=year, m=month, pl = plan)
    
def get_output_file(plan = "Prepago"):
    output = "/home/juan/mobility-study/mexico-scripts-ver2/output/raw_antennas.csv"
    return output


## small test

In [25]:
months = range(1,6)
month = random.choice(months)
print(month)

sframe_dir = get_input_file(m,plan="Pospago")
sframe_dir

4


'/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/2015/05/Pospago'

In [17]:
# aca voy a ir guardando las antennas ya separadas
output_antennas = gl.SFrame()

In [15]:
test =  gl.load_sframe(sframe_dir)
test.remove_columns(['USER','OTHER_USER','DURATION','DURATION','DATE'])

LATITUDE,LONGITUDE
19.515481,-98.8714
21.1622,-100.9297
19.07394444,-98.16555556
19.45922222,-99.18736111
32.412658,-116.960317
19.26961219,-103.7188352
19.64177778,-99.09097222
21.03872472,-89.56379139
19.533931,-99.174701
19.50204167,-99.12666667


In [13]:
test.shape

(265201623, 7)

In [19]:
%%time
#hago una agrupacion estupida pero en verdad esto es 
test2 = test.groupby(['LATITUDE','LONGITUDE'],  {'CEL_ID':gl.aggregate.COUNT('LONGITUDE')})

CPU times: user 7min 22s, sys: 3.84 s, total: 7min 26s
Wall time: 52.1 s


In [22]:
output_antennas = output_antennas.append(test2)

LATITUDE,LONGITUDE,CEL_ID
24.14206333,-110.2940553,4778
19.31787222,-99.13700278,77916
17.87924083,-92.48047778,100754
22.415,-97.938,2254
19.06744444,-98.221,24089
18.82805556,-99.24444444,150600
17.9997,-92.9282,163058
25.80656932,-100.3256526,49045
19.4906,-99.2712,179225
20.58142655,-103.4326277,109406


# ahora iteramos sobre todos los meses del 2015

In [32]:
#params
months = range(1,13)
plan = "Pospago"
months,year , plan

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2015, 'Pospago')

In [33]:
%%time
start_time = time.time()
local_time = time.localtime()

# aca voy a ir guardando las antennas ya separadas
output_antennas = gl.SFrame()

print("Read all Sframe dirs and groupby and append to one antennas table, time elapsed is {t}  localtime is {ho}:{mi:0=2d}"\
          .format(t=(time.time()-start_time), ho = local_time.tm_hour , mi = local_time.tm_min ))

for m in months:
    
    local_time = time.localtime()
    print("Process month {m}, time elapsed is {t}  localtime is {ho}:{mi:0=2d}"\
          .format(m = m,t=(time.time()-start_time), ho = local_time.tm_hour , mi = local_time.tm_min ))
    sframe_dir = get_input_file(m,plan="Pospago")
    print(sframe_dir)
    #hago una agrupacion estupida pero en verdad esto es 
    
    monthly_table =  gl.load_sframe(sframe_dir)
    monthly_table.remove_columns(['USER','OTHER_USER','DURATION','DURATION','DATE'])
    
    local_time = time.localtime()
    print("Group sframe table time elapsed is {t}  localtime is {ho}:{mi:0=2d}"\
          .format(t=(time.time()-start_time), ho = local_time.tm_hour , mi = local_time.tm_min ))
    monthly_table = monthly_table.groupby(['LATITUDE','LONGITUDE'],  {'CEL_ID':gl.aggregate.COUNT('LONGITUDE')})

    local_time = time.localtime()
    print("Append processed month {m}, time elapsed is {t}  localtime is {ho}:{mi:0=2d}"\
          .format(m = m,t=(time.time()-start_time), ho = local_time.tm_hour , mi = local_time.tm_min ))
    output_antennas = output_antennas.append(monthly_table)
    
#tomo la agrupac

Read all Sframe dirs and groupby and append to one antennas table, time elapsed is 0.00118684768677  localtime is 2:16
Process month 1, time elapsed is 0.00141382217407  localtime is 2:16
/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/2015/01/Pospago
Group sframe table time elapsed is 0.0423078536987  localtime is 2:16
Append processed month 1, time elapsed is 255.074151993  localtime is 2:20
Process month 2, time elapsed is 255.077152967  localtime is 2:20
/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/2015/02/Pospago
Group sframe table time elapsed is 255.109897852  localtime is 2:20
Append processed month 2, time elapsed is 504.548683882  localtime is 2:24
Process month 3, time elapsed is 504.55141902  localtime is 2:24
/home/juan/mobility-study/mexico-scripts-ver2/sframe_cdrs/2015/03/Pospago
Group sframe table time elapsed is 504.586169004  localtime is 2:24
Append processed month 3, time elapsed is 879.284766912  localtime is 2:30
Process month 4, time elapse

In [34]:
print(output_antennas.shape)

(51107, 3)


# hago analytics simple antes de agrupar

In [35]:
output_antennas.head()

LATITUDE,LONGITUDE,CEL_ID
24.14206333,-110.2940553,6418
19.31787222,-99.13700278,70792
17.87924083,-92.48047778,95416
22.415,-97.938,2286
19.06744444,-98.221,20494
18.82805556,-99.24444444,138004
17.9997,-92.9282,148980
25.80656932,-100.3256526,59709
19.4906,-99.2712,157177
20.58142655,-103.4326277,95170


In [36]:
%%time
count_nulls = output_antennas[ output_antennas['CEL_ID'] == None ]
print(count_nulls.shape)

(0, 3)
CPU times: user 328 ms, sys: 168 ms, total: 496 ms
Wall time: 502 ms


In [37]:
#count same lat/long
count_same = output_antennas[ output_antennas['LONGITUDE'] == output_antennas['LATITUDE'] ]
print(count_same.shape)

(0, 3)


# Finalmente agrupo y surrogo las antennas

In [38]:
output_antennas = output_antennas.groupby(['LATITUDE','LONGITUDE'],  {'CEL_ID':gl.aggregate.COUNT('LONGITUDE')})
output_antennas['CEL_ID'] = range(1,output_antennas.shape[0]+1)

In [39]:
output_antennas.shape

(4415, 3)

In [42]:
output_antennas.sample(0.01)

LATITUDE,LONGITUDE,CEL_ID
28.69472222,-106.1080556,55
27.45688333,-99.50775,331
19.49628056,-98.83612222,372
25.588897,-100.280822,382
19.45413889,-99.20716667,386
19.226222,-96.217667,414
19.408472,-99.171889,506
24.76055556,-107.4358333,627
25.78763889,-100.1445806,634
19.435722,-99.231778,692


# save to output

In [46]:
output_file = get_output_file("Pospago")
output_antennas.export_csv(output_file, delimiter="|")

In [47]:
!wc -l $output_file
!du -ha $output_file
!head $output_file

4416 /home/juan/mobility-study/mexico-scripts-ver2/output/raw_antennas.csv
96K	/home/juan/mobility-study/mexico-scripts-ver2/output/raw_antennas.csv
LATITUDE|LONGITUDE|CEL_ID
24.1421|-110.294|1
19.3179|-99.137|2
17.8792|-92.4805|3
22.415|-97.938|4
19.0674|-98.221|5
18.8281|-99.2444|6
17.9997|-92.9282|7
25.8066|-100.326|8
19.4906|-99.2712|9


## Mata las colas de los floats, pruebo con pandas

In [50]:
import pandas as pd
output_antennas2 =  output_antennas.to_dataframe()

In [51]:
output_antennas2.head()

,LATITUDE,LONGITUDE,CEL_ID
0,24.142063,-110.294055,1
1,19.317872,-99.137003,2
2,17.879241,-92.480478,3
3,22.415000,-97.938000,4
4,19.067444,-98.221000,5


In [53]:
output_antennas2.to_csv(output_file, sep="|",index=False,header=True)

In [54]:
!wc -l $output_file
!du -ha $output_file
!head $output_file

4416 /home/juan/mobility-study/mexico-scripts-ver2/output/raw_antennas.csv
120K	/home/juan/mobility-study/mexico-scripts-ver2/output/raw_antennas.csv
LATITUDE|LONGITUDE|CEL_ID
24.14206333|-110.2940553|1
19.31787222|-99.13700278|2
17.87924083|-92.48047778|3
22.415|-97.938|4
19.06744444|-98.221|5
18.82805556|-99.24444444|6
17.9997|-92.9282|7
25.80656932|-100.3256526|8
19.4906|-99.2712|9
